In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
import config.py

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL").config("spark.driver","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Import struct fields that we can use
from pyspark.sql.types import *

In [ ]:
schema = [StructField("marketplace", StringType(), True),
          StructField("customer_id", IntegerType(), True),
          StructField("review_id", StringType(), True),
          StructField("product_id", IntegerType(), True),
          StructField("product_parent", IntegerType(), True),
          StructField("product_title", StringType(), True),
          StructField("product_category", StringType(), True),
          StructField("star_rating", IntegerType(), True),
          StructField("helpful_votes", IntegerType(), True),
          StructField("total_votes", IntegerType(), True),
          StructField("vine", StringType(), True),
          StructField("verified_purchase", StringType(), True),
          StructField("review_headline", StringType(), True),
          StructField("review_body", StringType(), True),
          StructField("review_date", DateType(), True)]

final = StructType(fields=schema)

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), sep="\t", header=True,schema=final)

# Show DataFrame
df.show()

In [ ]:
df.dtypes

In [ ]:
df.count()

In [ ]:
from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window

df_with_seq_id = df.withColumn('customer_count', row_number().over(Window.orderBy(monotonically_increasing_id())))

In [ ]:
df_with_seq_id.agg({'customer_count': 'max'}).show()

In [ ]:
review_id_table=df.select(['review_id','customer_id','product_id','product_parent','review_date'])
products=df.select(['product_id','product_title'])
customers=df_with_seq_id.select(['customer_id','customer_count'])
vine_table=df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])

In [ ]:
# Configure settings for RDS
mode = "overwrite"
jdbc_url="jdbc:postgresql://localhost:5433/Big_Data_2"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)